# Use Case 2: Creating buildings from Overpass turbo API query

## Create a query
Shown below is a query for a some buildings in Bedburg:
```json
[out:json];
(
  way["building"](poly:"51.009961 6.551451 51.010646 6.550679 51.010329 6.549949 51.009559 6.550722");
);
out body;
>;
out skel qt;
```

If you are using [overpass turbo](https://overpass-turbo.eu/) you can use the map to make sure you got the right area and all wanted buildings are showing up. If you switch to the "data" tab you get your query result as a json file which should something like this:

```json
{
    "version": 0.6,
    "generator": "Overpass API 0.7.62.1 084b4234",
    "osm3s": {
        "timestamp_osm_base": "2024-09-18T14:03:30Z",
        "copyright": "The data included in this document is from www.openstreetmap.org. The data is made available under ODbL."
    },
    "elements": [
        {
            "type": "way",
            "id": 1194286054,
            "nodes": [
                11085711206,
                11085711205,
                11085711204,
                11085711203,
                11085711206
            ],
            "tags": {
                "building": "garage"
            }
        },
...
        {
            "type": "node",
            "id": 11085711297,
            "lat": 51.0099692,
            "lon": 6.5505769
        }
    ]
}
```
with elements of type "nodes" representing coordinates and "way" representing buildings.
We safe the results to a ```exampleQeury.json``` file.

## Code

### Importing need libraries

In [1]:
from citydpc import Dataset
import json
from citydpc.tools import cityBIT
from citydpc.core.output import cityjsonOutput
from pyproj import Transformer

### Loading query

In [2]:
with open("exampleQuery.json") as f:
    osmQuery = json.load(f)

We also create an empty CityDPC dataset and a coordinate transformer to transform the query results from epsg:4325 to our local coordinate system epsg:5555.

In [3]:
dataset = Dataset()
t = Transformer.from_crs("epsg:4326", "epsg:5555")

## Creating building objects

In [4]:
# split query into dict of nodes and ways for easier access
nodes = {}
ways = {}
for element in osmQuery["elements"]:
    if element["type"] == "node":
        nodes[element["id"]] = element
    elif element["type"] == "way":
        ways[element["id"]] = element

# transform all nodes to local coordinate system
for node in nodes.values():
    lat, lon = t.transform(node["lat"], node["lon"])
    node["lat"] = round(lat, 3)
    node["lon"] = round(lon, 3)

# iterate over ways and create building objects
for bID, obj in ways.items():
    coordinates = []
    # iterate over nodes and append them to the coordinates list
    for node in obj["nodes"]:
        coordinates.append([nodes[node]["lat"], nodes[node]["lon"]])
    # create building object with a ground surface height of 160m and 
    # building height of 10m with a flat roof (code: "1000")
    buildingHeight = 10
    if obj["tags"]["building"] == "garage":
        buildingHeight = 4
    building = cityBIT.create_LoD2_building(str(bID), coordinates, 160, buildingHeight, "1000")
    # add building object to dataset
    dataset.add_building(building)


## Moving created buildings using CityGTV

In [5]:
from citydpc.tools import cityGTV

# transform dataset (only moving with dataset in this case)
movedDataset = cityGTV.transform_dataset(
    dataset,
    "epsg:5555",
    "epsg:5555",
    (360133.335, 5706839.754),
    (324219.701, 5641681.61),
    rotAngle=0,
    eleChange=0,
)

## Saving dataset to CityJSON

In [6]:
# call function for export and set new file name
cityjsonOutput.write_cityjson_file(movedDataset, "transformed.gml", referenceSystem="https://www.opengis.net/def/crs/EPSG/0/5555")